1.   Создать Dataset для загрузки данных (используем только числовые данные)
2.   Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели. 

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm_notebook
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
data = pd.read_csv('/content/drive/MyDrive/PyTorch_Intro/train.csv')
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [39]:
# Adding day/month/week attributes
data.activation_date = pd.to_datetime(data.activation_date)
data['day_of_month'] = data.activation_date.apply(lambda x: x.day)
data['day_of_week'] = data.activation_date.apply(lambda x: x.weekday())

In [40]:
# Adding average probability per aggregated groups
groups = ['region', 'category_name',
            'city', 'user_type', 'parent_category_name']

for col in tqdm_notebook(groups):
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    p = data.groupby(col)['deal_probability']
    mean = p.mean()
    data[col + '_prob_avg'] = data[col].map(mean)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/5 [00:00<?, ?it/s]

In [41]:
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
data['region_1'] = labelencoder.fit_transform(data['region'])
data['city_1'] = labelencoder.fit_transform(data['city'])
data['category_name_1'] = labelencoder.fit_transform(data['category_name'])
data['user_type_1'] = labelencoder.fit_transform(data['user_type'])

data.head()

In [42]:
# Dropping non-numeric columns 
data = data.drop(['item_id', 'user_id', 'description', 'image', 'parent_category_name',
                    'param_1', 'param_2', 'param_3', 'title', 'activation_date'], axis=1)

In [43]:
data.head()

,region,city,category_name,price,item_seq_number,user_type,image_top_1,deal_probability,day_of_month,day_of_week,region_prob_avg,category_name_prob_avg,city_prob_avg,user_type_prob_avg,parent_category_name_prob_avg
0,19,460,42,400.0,2,1,1008.0,0.12789,28,1,0.122004,0.198445,0.123397,0.149557,0.075876
1,17,1300,22,3000.0,19,1,692.0,0.00000,26,6,0.136721,0.191848,0.139400,0.149557,0.179633
2,16,1276,2,4000.0,9,1,3032.0,0.43177,20,0,0.135944,0.171572,0.124881,0.149557,0.175421
3,21,940,42,2200.0,286,0,796.0,0.80323,25,5,0.142602,0.198445,0.135031,0.124513,0.075876
4,4,317,0,40000.0,3,1,2264.0,0.20797,16,3,0.145908,0.278427,0.137275,0.149557,0.263336


In [44]:
# Replacing None with medians
for col in data.columns:
    if data[col].isna().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)

In [45]:
display(data.head(2), data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   region                         1503424 non-null  int64  
 1   city                           1503424 non-null  int64  
 2   category_name                  1503424 non-null  int64  
 3   price                          1503424 non-null  float64
 4   item_seq_number                1503424 non-null  int64  
 5   user_type                      1503424 non-null  int64  
 6   image_top_1                    1503424 non-null  float64
 7   deal_probability               1503424 non-null  float64
 8   day_of_month                   1503424 non-null  int64  
 9   day_of_week                    1503424 non-null  int64  
 10  region_prob_avg                1503424 non-null  float64
 11  category_name_prob_avg         1503424 non-null  float64
 12  city_prob_avg 

,region,city,category_name,price,item_seq_number,user_type,image_top_1,deal_probability,day_of_month,day_of_week,region_prob_avg,category_name_prob_avg,city_prob_avg,user_type_prob_avg,parent_category_name_prob_avg
0,19,460,42,400.0,2,1,1008.0,0.12789,28,1,0.122004,0.198445,0.123397,0.149557,0.075876
1,17,1300,22,3000.0,19,1,692.0,0.00000,26,6,0.136721,0.191848,0.139400,0.149557,0.179633


None

In [46]:
# Splitting on train and test with stated params
train, test = train_test_split(data, test_size=0.25, random_state=13)
train.head(2)

,region,city,category_name,price,item_seq_number,user_type,image_top_1,deal_probability,day_of_month,day_of_week,region_prob_avg,category_name_prob_avg,city_prob_avg,user_type_prob_avg,parent_category_name_prob_avg
31723,9,755,9,1300.0,6,1,1056.0,0.0,23,3,0.147066,0.178480,0.189069,0.149557,0.263336
929954,19,460,29,1300.0,16,1,399.0,0.0,28,1,0.122004,0.046447,0.123397,0.149557,0.075876


In [47]:
class AvitoDataset(torch.utils.data.Dataset):
    def __init__(self, df, normalize=False, fit_scaler=False):
        self.df = df.copy()
        self.normalize = normalize
        self.scaler = MinMaxScaler()
        self.fit_scaler = fit_scaler
        self.sc_fl = 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx, -1:]
        if self.fit_scaler:
            self.scaler.fit(df.iloc[:, :-1])
            self.sc_fl = 1

        if self.normalize and sc_fl:
            df = scaler.transform(df)

        tensor = torch.FloatTensor(self.df.iloc[idx, 1:-1].values)
        label = torch.FloatTensor(label.values)

        return tensor, label

In [48]:
train_dataset = AvitoDataset(train)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=2048,
                                           shuffle=True,
                                           num_workers=3)

test_dataset = AvitoDataset(test)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=2048,
                                          shuffle=True,
                                          num_workers=3)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [49]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 5*hidden_dim)
        self.dp1 = nn.Dropout(0.40)

        self.bn2 = nn.BatchNorm1d(5*hidden_dim)
        self.fc2 = nn.Linear(5*hidden_dim, 2*hidden_dim)
        self.dp2 = nn.Dropout(0.15)

        self.bn4 = nn.BatchNorm1d(2*hidden_dim)
        self.fc4 = nn.Linear(2*hidden_dim, 1)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.dp1(x)

        x = self.bn2(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.dp2(x)

        x = self.bn4(x)
        x = self.fc4(x)
        x = torch.sigmoid(x)

        return x

In [50]:
def rmsle_loss(y_pred, y_true):
    loss = torch.sqrt(torch.mean((torch.log(y_pred+1)-torch.log(y_true+1))**2))
    return loss

In [51]:
def train_loop(tr_dataloader, ev_dataloader, model, optimizer, history):

    size = len(tr_dataloader.dataset)
    for batch, (X, y) in enumerate(tr_dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = rmsle_loss(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            current = batch * len(X)
            history['train'].append(loss)
            test_loss = eval_loop(ev_dataloader, model)
            history['eval'].append(test_loss)
            print(f"loss: {loss:>7f}, Avg test loss: {test_loss:>8f}  [{current:>5d}/{size:>5d}]")

    return history


def eval_loop(ev_dataloader, model):
    
    size = len(ev_dataloader.dataset)
    num_batches = len(ev_dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in ev_dataloader:
            pred = model(X)
            test_loss += rmsle_loss(pred, y)

    test_loss /= num_batches
    
    return test_loss

ADAM

In [52]:
net = FeedForward(13, 5)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [53]:
history_1 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(2)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_1 = train_loop(train_loader,
                           test_loader,
                           net,
                           optimizer,
                           history_1
                          )
print("Training is finished!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


loss: 0.317848, Avg test loss: 0.305285  [    0/1127568]
loss: 0.051061, Avg test loss: 0.050677  [102400/1127568]
loss: 0.031746, Avg test loss: 0.031888  [204800/1127568]
loss: 0.029527, Avg test loss: 0.029196  [307200/1127568]
loss: 0.027121, Avg test loss: 0.027802  [409600/1127568]
loss: 0.027032, Avg test loss: 0.026975  [512000/1127568]
loss: 0.025204, Avg test loss: 0.026717  [614400/1127568]
loss: 0.025965, Avg test loss: 0.026250  [716800/1127568]
loss: 0.026004, Avg test loss: 0.025824  [819200/1127568]
loss: 0.025843, Avg test loss: 0.025554  [921600/1127568]
loss: 0.026809, Avg test loss: 0.025313  [1024000/1127568]
loss: 0.024840, Avg test loss: 0.025221  [642400/1127568]
Epoch 2
-------------------------------
loss: 0.025572, Avg test loss: 0.025305  [    0/1127568]
loss: 0.025586, Avg test loss: 0.025258  [102400/1127568]
loss: 0.025158, Avg test loss: 0.024971  [204800/1127568]
loss: 0.024889, Avg test loss: 0.024974  [307200/1127568]
loss: 0.025186, Avg test loss: 0.

SGD

In [54]:
net = FeedForward(13, 5)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [55]:
history_2 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(1 )):
    print(f"Epoch {t+1}\n-------------------------------")
    history_2 = train_loop(train_loader,
                           test_loader,
                           net,
                           optimizer,
                           history_2
                          )
print("Training is finished!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


loss: 0.273137, Avg test loss: 0.272373  [    0/1127568]
loss: 0.258749, Avg test loss: 0.257059  [102400/1127568]
loss: 0.244548, Avg test loss: 0.241631  [204800/1127568]
loss: 0.225242, Avg test loss: 0.226149  [307200/1127568]
loss: 0.209972, Avg test loss: 0.210885  [409600/1127568]
loss: 0.195315, Avg test loss: 0.195881  [512000/1127568]
loss: 0.182224, Avg test loss: 0.181321  [614400/1127568]
loss: 0.168836, Avg test loss: 0.167268  [716800/1127568]
loss: 0.154960, Avg test loss: 0.153878  [819200/1127568]
loss: 0.139495, Avg test loss: 0.141229  [921600/1127568]
loss: 0.128354, Avg test loss: 0.129315  [1024000/1127568]
loss: 0.115984, Avg test loss: 0.118196  [642400/1127568]
Training is finished!


RMSProp

In [56]:
net = FeedForward(13, 5)

optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01)

In [57]:
history_3 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(1)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_3 = train_loop(train_loader,
                           test_loader,
                           net,
                           optimizer,
                           history_3
                          )
print("Training is finished!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


loss: 0.297542, Avg test loss: 0.261415  [    0/1127568]
loss: 0.036190, Avg test loss: 0.035916  [102400/1127568]
loss: 0.029332, Avg test loss: 0.028749  [204800/1127568]
loss: 0.028664, Avg test loss: 0.028887  [307200/1127568]
loss: 0.025104, Avg test loss: 0.025828  [409600/1127568]
loss: 0.024736, Avg test loss: 0.026496  [512000/1127568]
loss: 0.025868, Avg test loss: 0.025998  [614400/1127568]
loss: 0.026543, Avg test loss: 0.026441  [716800/1127568]
loss: 0.024065, Avg test loss: 0.024905  [819200/1127568]
loss: 0.024852, Avg test loss: 0.024714  [921600/1127568]
loss: 0.024677, Avg test loss: 0.024618  [1024000/1127568]
loss: 0.024838, Avg test loss: 0.024533  [642400/1127568]
Training is finished!


При одинаковых параметрах модели для 1 эпохи - скорость схождения быстрее у RMSProp, почти такой же результат у оптимизатора Adam. Больше всего потери после 1 эпохи у SGD.